In [8]:
import requests
from bs4 import BeautifulSoup
import re
import os
from pathlib import Path

from tqdm.notebook import tqdm

In [9]:
def make_dir(dir_path):
    if not dir_path.is_dir():
        os.mkdir(dir_path)
        print(f'Created: {dir_path.as_posix()}')
    else:
        print(f"{dir_path.as_posix()} exists")

In [10]:
def get_txt_script(script_url):
    '''
    Input is just the html page, without action added.
    return the text of the episode script. 
    Always starts with 5 +. And ends with 5 +
    Scenes are separated by also 5 +
    '''
    raw_html = requests.get(script_url)

    soup = BeautifulSoup(raw_html.text, 'html.parser')

    # Get the (one) box of the script links information
    try:
        main_table = soup.find_all('table', class_='headerscontent')[1]
    except IndexError as e:
        print(e)
        print(script_url)
        main_table = soup.find_all('table', class_='headerscontent')     
        text = ''
        for table in main_table:
            mids = main_table.find_all('td', class_='DLborderBOT')
            
            for mid in mids:
                if len(mid.text)<=2:
                    text += '+'*5 + '\n'
                text += mid.text + '\n'
            text += '+'*5

        return text
        
    
#     for table in main_table[1:]: # From table 1
#     rights = table.find_all('td', class_='DLborderRIGHT')
    mids = main_table.find_all('td', class_='DLborderBOT')

    text = ''
    for mid in mids:
        if len(mid.text)<=2:
            text += '+'*5 + '\n'
        text += mid.text + '\n'
    text += '+'*5

    return text

Create the directory to store all the scripts in.

In [11]:
current_dir = Path.cwd() # So that I know where I start

# Set the path of the directory in which the scripts for each season and each episode will be stored in
scripts_path = current_dir / 'Scripts'

# Create the Scripts dir
make_dir(scripts_path)

Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts


Now I can get the links to all the seasons to search in for episodes and finally scripts.

Url of the [scripts portal](https://southpark.fandom.com/wiki/Portal:Scripts). Lists all the seasons.

In [12]:
# Set the base url. Used to add to it
base_url = r'https://southpark.fandom.com/wiki/'

# For the 'API', when we want to get the wikisource style page. It is easier to parse
action = r'?action=edit' 

# Create the url to get the seasons links
scripts_main_url = base_url+ r'Portal:Scripts' + action
raw_scripts_main_html = requests.get(scripts_main_url)
soup_season = BeautifulSoup(raw_scripts_main_html.text, 'html.parser')

# Get the (one) box of the script links information
main_box_seasons = soup_season.find_all('textarea', id='wpTextbox1')[0] # It always has 1 box

In [13]:
# Use regex to get all the seasons links in a list. This pattern will be used
# for the seasons links and then the episodes links.
# pattern = r'\|link\=(.+)\n'

pattern = r"\|link\=([^\|\n]+)\n"

# Create a dictionary for the links. It is a dict of dicts. Keys are season numbers and values are dictionaries with keys episode
# and link as value
scripts_dict = {}

Now start going inside every season then every episode and finally inside each script. Get its text and store it in a text file in the corresponding folder.

In [14]:
season = 1
matches_season = re.findall(pattern , main_box_seasons.get_text())
for match_season in tqdm(matches_season):
    # Get the link and replace space with _. Not essential, automatically done
    link_name_seas = match_season.replace(' ', '_')
    
    # Get the name of the season 
#     ses_dir_name = link_name.split('/')[-1]
    ses_dir_name = str(season)
    if season <10:
        ses_dir_name = '0'+ses_dir_name
        
    
    # Directory to create
    season_dir_path = scripts_path / ses_dir_name
    make_dir(season_dir_path)
    
    ## Get all the episodes in this season. Similar procedure
    episodes_url = base_url + link_name_seas + action
    episodes_html = requests.get(episodes_url)
    soup_episodes = BeautifulSoup(episodes_html.text, 'html.parser')
    
    # Get the (one) box of the script links information
    main_box_episodes = soup_episodes.find_all('textarea', id='wpTextbox1')[0] # It always has 1 box
    episode = 1
    matches_episodes = re.findall(pattern , main_box_episodes.get_text())
        
    for match_episode in matches_episodes:
        # Get the link and replace space with _. Not essential, automatically done
        link_name_ep = match_episode.replace(' ', '_')
                
        # Get the name of the episode. Add its number in the beginning 
        fname = link_name_ep.split('/')[-2].replace('?', '').replace('\'', '').replace('.', '') # It has script added in the end
        fname = fname.replace(':', "_")
        ep_dir_name = str(episode) + "_" + fname + '.txt'
        if episode<10:
            ep_dir_name = str(0)+ep_dir_name
        
        # Directory to create
        episode_dir_path = season_dir_path / ep_dir_name
        Path(episode_dir_path).touch()
        
        
        script_link = base_url + link_name_ep
        with open(episode_dir_path.as_posix(), 'w', encoding='utf-8') as f:
            f.write(get_txt_script(script_link.replace('?', '%3F')))
        
        
        # Increment episode
        episode += 1
        
    
        # Update dict
        scripts_dict[season] = {episode: script_link}
    
    # Increment season number
    season += 1

  0%|          | 0/25 [00:00<?, ?it/s]

Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/01
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/02
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/03
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/04
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/05
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/06
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/07
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social graphs/g_panag_repo/projects_socialgraphs22/Project/Scripts/08
Created: C:/Users/user/Documents/DTU/Fall_22_23/Social g